<a href="https://colab.research.google.com/github/leandroaguazaco/data_science_portfolio/blob/main/Projects/04-Churn_Telco_Analysis/04_Churn_Telco_Analysis_02_Premodeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"> 4 - CHURN TELCO ANALYSIS </h1>

<div align="center">

  <img alt="Static Badge" src="https://img.shields.io/badge/active_project-true-blue">

</div>  

<h2 align="center"> 4.1 - Premodeling </h2>

<div align="center">

  <img alt="Static Badge" src="https://img.shields.io/badge/active_section-true-blue">

  <img alt="Static Badge" src="https://img.shields.io/badge/section_status-in progress-green">

</div>  

<object
data="https://img.shields.io/badge/contact-Felipe_Leandro_Aguazaco-blue?style=flat&link=https%3A%2F%2Fwww.linkedin.com%2Fin%2Ffelipe-leandro-aguazaco%2F">
</object>

## a. Project summary

The aim of this project is to analyze and predict customer churn in the telco industry. The information pertains to client behavior, including in-call, out-call, and internet service consumption. There is a variable called 'Churn' that determines whether a customer churned within two weeks after canceling services. The information summarizes eight weeks of data for each telco line or client.

<h3 align="center"> <font color='orange'>NOTE: The project is distributed across multiple sections, separated into notebook files, in the following way:</font> </h3>



4.1 - Preprocessig data: load, join and clean data, and Exploratory data analysis, EDA.

> <font color='gray'> 4.2 - Premodeling: predict customer churn based on PyCaret library. </font> ✍ ▶ <font color='orange'> Current section </font>

4.3 - Modeling: predict customer churn based on sklearn pipelines.

4.4 - Analyzing and explaining predictions.

4.5 - Detecting vulneabilities in final machine learnig model.

4.6 - Model deployment with Streamlit.

## b. Install libraries

In [ ]:
%%capture
!pip install -U --pre pycaret
!pip install rpy2==3.5.1 # Use R
!pip install colorama

## c. Import libraries

In [ ]:
%%capture
# c.1 Python Utilies
import pandas as pd
import numpy as np
import rpy2
import shutil
from google.colab import drive
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from colorama import Fore, Style

In [ ]:
# c.2 PyCaret
from pycaret.classification import *
from pycaret.regression import *

In [ ]:
# c.3 Setups
%matplotlib inline
plt.style.use("ggplot")
warnings.simplefilter("ignore")

## d. Custom functions

### d.1 - Type conversions

In [ ]:
# d.1 dtypes conversion and memory reduce function.
def dtype_conversion(df: pd.DataFrame = None, verbose: bool = True)-> pd.DataFrame:
    """
    Summary:
      Function to dtypes conversion and save reduce memory usage; takes a DataFrame as argument, returns DataFrame.
      For more details, visit: https://towardsdatascience.com/how-to-work-with-million-row-datasets-like-a-pro-76fb5c381cdd.
      The modifications include type casting for numerical and object variables.
    Parameters:
      df (pandas.DataFrame): DataFrame containing information.
      verbose (bool, default = True): If true, display results (conversions and warnings)
    Returns:
      pandas.DataFrame: original DataFrame with dtypes conversions
      Plot original dtypes status, variable warning due high cardinality, save memory usage, final dtypes status.
    """
    # 0- Original dtypes
    # print(Fore.GREEN + "Input dtypes" + Style.RESET_ALL)
    # print(df.dtypes)
    # print("\n")
    print(Fore.RED + "High Cardinality, categorical features with levels > 15" + Style.RESET_ALL)

    # 1- Original memory_usage in MB
    start_mem = df.memory_usage().sum() / 1024 ** 2

    # 2- Numerical Types
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int": # First 3 characters
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                #elif (c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max):
                #    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    # 3- Categorical Types
    high_card_vars = 0
    for col in df.select_dtypes(exclude = ["int8", "int16", "int32", "int64", "float16", "float32", "float64", "datetime64[ns]"]):
        categories = list(df[col].unique())
        cat_len = len(categories)
        if cat_len >= 2 and cat_len < 15:
           df[col] = df[col].astype("category")
        else:
          high_card_vars =+ 1
          # Print hight cardinality variables, amount of levels and a sample of 50 firts categories
          print(f"Look at: {Fore.RED + col + Style.RESET_ALL}, {cat_len} levels = {categories[:50]}")
    if high_card_vars == 0:
      print(Fore.GREEN + "None" + Style.RESET_ALL)
    else:
      pass

    # 4- Final memory_usage in MB
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print("\n")
        print(f"{Fore.RED}Initial memory usage: {start_mem:.2f} MB{Style.RESET_ALL}")
        print(f"{Fore.BLUE}Memory usage decreased to {end_mem:.2f} MB ({ 100 * (start_mem - end_mem) / start_mem:.1f}% reduction){Style.RESET_ALL}")
        #print("\n")
        #print(Fore.GREEN + "Output dtypes" + Style.RESET_ALL)
        #print(df.dtypes)
        print("\n")

    # 5. Feature types
    print(Fore.GREEN + "Variable types" + Style.RESET_ALL)
    numerical_vars = len(df.select_dtypes(include = ["number"]).columns)
    categorical_vars = len(df.select_dtypes(include = ["category", "object"]).columns)
    datetime_vars = len(df.select_dtypes(include = ["datetime64[ns]"]).columns)
    print(f"Numerical Features: {numerical_vars}")
    print(f"Categorical Features: {categorical_vars}")
    print(f"Datetime Features: {datetime_vars}")

    return df

## 1 - Data

### 1.1 - Import from Google Drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')


# Specify the source path in Google Drive
drive_filepath = '/content/drive/MyDrive/DataScience_Portfolio/04-Churn_Telco_Analysis/'

# Specify the destination path in Colab
colab_filepath = '/content/'

# Copy the file from Google Drive to Colab
try:
  shutil.copy(src = drive_filepath + '/churn_data.txt', dst = colab_filepath + '/churn_data.txt')
except:
  pass

Mounted at /content/drive


### 1.2 - Load data

In [ ]:
churn_df = pd.read_csv(filepath_or_buffer = "churn_data.txt",
                       sep = "|",
                       index_col = "SUBSCRIBER_ID",
                       parse_dates = True,
                       decimal = ".",
                       encoding = "utf-8") \
             .assign(region = lambda x: x.loc[:, "region"].astype("category")) \
             .pipe(dtype_conversion) \
             .dropna(subset = ["churn"]) \
             .reset_index(drop = True)

# Security copy
churn_df_copy = churn_df.copy(deep = True)

High Cardinality, categorical features with levels > 15
Look at: region, 34 levels = ['Bogota D.C.', 'Santander', 'Antioquia', nan, 'Cundinamarca', 'Quindio', 'Valle Del Cauca', 'Arauca', 'Bolivar', 'Atlantico', 'Tolima', 'Huila', 'Meta', 'Putumayo', 'Boyaca', 'Caldas', 'Cordoba', 'Nariño', 'Magdalena', 'Risaralda', 'Cauca', 'La Guajira', 'Norte De Santander', 'Caqueta', 'Cesar', 'Guaviare', 'Sucre', 'Amazonas', 'Guainia', 'Vichada', 'Choco', 'Casanare', 'Providencia Islas', 'Vaupes']


Initial memory usage: 196.19 MB
Memory usage decreased to 92.12 MB (53.0% reduction)


Variable types
Numerical Features: 38
Categorical Features: 5
Datetime Features: 0


## 2 - Predicting Customer Churn

Pycaret
>[Home - Pycaret](https://pycaret.org/)

>[Read the docs - Pycaret](https://pycaret.readthedocs.io/en/stable/index.html)

### 2.1 - Init setup

In [ ]:
%%time

# 2.1 Init the experimentation class
churn_binaryclass = ClassificationExperiment()

# 2.2 Init setup

churn_binaryclass.setup(data = churn_df,
                        target = 'churn', # Target column in data
                        index = False, # Hold original index
                        session_id = 123,
                        preprocess = True,
                        verbose = True, # When set to False, Information grid is not printed.
                        train_size = 0.75, # Proportion of the dataset to be used for training and validation
                        test_data = None,

                        # Dtypes categorical features
                        ordinal_features = None,
                        categorical_features = ["canal", "region", 'bandas', 'tipo_gross_adds'],
                        max_encoding_ohe = 30, # Categorical columns with max_encoding_ohe or less unique values are encoded using OneHotEncoding
                        rare_to_value = 0.05, # Minimum fraction of category occurrences in a categorical column
                        rare_value = "Otro", # Value with which to replace rare categories

                        # Dtypes numerical features
                        numeric_features = list(churn_df.select_dtypes(include = 'number').columns),

                        # Dtypes datetime features
                        date_features = None,
                        create_date_columns = ["day", "month", "year"], # Columns to create from the date features

                        # Normalize data: depending on similarity of numerical variables scales
                        normalize = True,
                        normalize_method = "robust", # z-score, minmax, maxabs, robust

                        # Transform data: depending on skewnees, kurtosis, outiler presence
                        transformation = True,
                        transformation_method = "yeo-johnson",

                        # Handled missing values
                        imputation_type = "iterative",
                        iterative_imputation_iters = 5,
                        numeric_iterative_imputer = "lightgbm", #"lightgbm", # Default method
                        categorical_iterative_imputer = "lightgbm", # Default method

                        # Outliers
                        remove_outliers = False,
                        outliers_method = "ee", # "iforest": Uses sklearn’s IsolationForest; "ee": Uses sklearn’s EllipticEnvelope; "lof": Uses sklearn’s LocalOutlierFactor
                        outliers_threshold = 0.05, # Percentage of outliers to be removed from the dataset

                        # Imbalance data: depending on levels distributions in target variable
                        fix_imbalance = True, # Dataset has unequal distribution of target class it can be balanced using this parameter
                        fix_imbalance_method = 'SMOTE', # Synthetic Minority Over-sampling Technique, choose from the name of an imblearn estimator

                        # Feature selection
                        feature_selection = True,
                        feature_selection_method = "classic", # "sequential": uses sklearn's SequentialFeatureSelector, "classic": uses sklearn's SelectFromModel
                        feature_selection_estimator = "lightgbm", # Classifier used to determine the feature importance
                        n_features_to_select = 0.5, # The maximum number of features to select with feature_selection
                        low_variance_threshold = 0.1, # Remove features with a training-set variance lower than the provided threshold
                        pca = False,

                        # Multicollinearity
                        remove_multicollinearity = True, # Features with the inter-correlations higher than the defined threshold are removed
                        multicollinearity_threshold = 0.85, # Minimum absolute Pearson correlation to identify correlated features
                        #remove_perfect_collinearity = True,

                        # Cross validation stragery
                        data_split_shuffle = True,
                        data_split_stratify = True, # Controls stratification during train_test_split
                        fold_strategy = "stratifiedkfold",
                        fold = 5, # Number of folds to be used in cross validation, use in tuning hyperparameters

                        # Experiment Logging - Mlflow
                        #experiment_name = "Predicting NPS Type - v0.1",
                        #log_experiment = True,
                        #log_plots = True
)

,Description,Value
0,Session id,123
1,Target,churn
2,Target type,Binary
3,Target mapping,"No: 0, Sí: 1"
4,Original data shape,"(595663, 43)"
5,Transformed data shape,"(1012800, 22)"
6,Transformed train set shape,"(863884, 22)"
7,Transformed test set shape,"(148916, 22)"
8,Numeric features,38
9,Categorical features,4


CPU times: user 6min 57s, sys: 21.1 s, total: 7min 18s
Wall time: 5min 29s


### 2.2 - Check init setup results

In [ ]:
# 2.2. Access all the variables created by the setup function, check all available config

churn_binaryclass.get_config()
churn_binaryclass.get_config("pipeline")
# churn_binaryclass.get_config("_ml_usecase")
# churn_binaryclass.get_config("is_multiclass")



# churn_binaryclass.get_config("X_train")
# churn_binaryclass.get_config("y_train")
# churn_binaryclass.get_config("X_train_transformed")
# churn_binaryclass.get_config("y_train_transformed").value_counts()

# churn_binaryclass.get_config("X_test")
# churn_binaryclass.get_config("y_test")
# churn_binaryclass.get_config("X_test_transformed")
# churn_binaryclass.get_config("y_test_transformed")

Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('iterative_imputer',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=IterativeImputer(add_indicator=False,
                                                                 cat_estimator=LGBMClassifier(boosting_type='gbdt',
                                                                                              class_weight=None,
                                                                                              colsampl...
                                                                                         random_state=None,
                                                                                         reg_alpha=0.0,
                                                                                         reg_lambda=0.0,
                                                                                         silent='warn',
                                                                                         subsample=1.0,
                                                                                         subsample_for_bin=200000,
                                                                                         subsample_freq=0),
                                                                importance_getter='auto',
                                                                max_features=21,
                                                                norm_order=1,
                                                                prefit=False,
                                                                threshold=-inf))),
                ('clean_column_names',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=CleanColumnNames(match='[\\]\\[\\,\\{\\}\\"\\:]+')))],
         verbose=False)

In [ ]:
# PyCaret Models
churn_binaryclass.models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


### 2.3 - Compare and select models

In [ ]:
# 2.3 Compare models: metrics are calculated using cross-validation on the training set
best_models = churn_binaryclass.compare_models(sort = "Recall",
                                               n_select = 1,
                                               turbo = True,
                                               budget_time = None,
                                               cross_validation = True,
                                               # include = ['lr', 'nb', 'xgboost', 'lightgbm', 'ada', 'rf']
                                               include = ['lightgbm'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7483,0.6355,0.4392,0.0588,0.1037,0.0480,0.0822,284.0180
nb,Naive Bayes,0.8133,0.6711,0.4066,0.0747,0.1262,0.0744,0.1089,18.9020
ada,Ada Boost Classifier,0.9602,0.6689,0.1302,0.2846,0.1784,0.1609,0.1741,102.1320
lightgbm,Light Gradient Boosting Machine,0.9676,0.7248,0.1000,0.5611,0.1696,0.1612,0.2272,32.0800
xgboost,Extreme Gradient Boosting,0.9675,0.7266,0.0993,0.5592,0.1686,0.1602,0.2260,320.7640
rf,Random Forest Classifier,0.9673,0.6892,0.0933,0.5402,0.1590,0.1507,0.2148,285.2460


Processing:   0%|          | 0/34 [00:00<?, ?it/s]

### 2.4 - Analyze models

In [ ]:
# 1. Results plots

sns.set_palette(palette = "viridis") # Viridis pallete
churn_binaryclass.evaluate_model(best_models[0], plot_kwargs = {'cmap': sns.color_palette('viridis')})

In [ ]:
# 2. Interpret Model
# Implemented based on the SHAP (Shapley Additive exPlanations)
# https://shap.readthedocs.io/en/latest/

plt.rcParams['image.cmap'] = 'viridis'
churn_binaryclass.interpret_model(best_models[0], plot = 'summary')

### 2.5 - Predictions

In [ ]:
# Predictions
# Model 1: Light Gradient Boosting Machine

# Model's hyperparameters
print(best_models[0])
print("\n")

# Testing dataset
test_data = pd.concat([churn_binaryclass.get_config("X_test")]) \
              .loc[:, list(churn_binaryclass.get_config("X_train").columns)]

# Calculate predictions
churn_binaryclass.predict_model(best_models[0],
                                data = test_data,
                                raw_score = True)

### 2.6 -  Hyperparameters tuning

In [ ]:
# Hyperparameters tuning

# lightgbmClassifier
tuned_lightgbm = churn_binaryclass.tune_model(estimator = best_models[0],
                                              optimize = "Recall",
                                              search_library = "scikit-optimize",
                                              search_algorithm = "bayesian")
print("\n")
print(tuned_lightgbm)

### 2.7 - Save and load models

In [ ]:
# Save models: create pickle file

# lightgbmClassifier
churn_binaryclass.save_model(model = tuned_lightgbm, model_name = 'LightgbmClassifier_model_churn')

In [ ]:
# Load the model back in environment
lightgbm_model_tuned = load_model('LightgbmClassifier_model_churn')

### 2.8 - Save / Load Experiment

In [ ]:
# 1. Save experiment
churn_binaryclass.save_experiment("churn_binaryclass_experiment")

In [ ]:
# 10. 2 Load experiment from disk
churn_binaryclass2 = load_experiment(path_or_file = 'churn_binaryclass_experiment',
                                     data = churn_df) # From step 1